In [373]:
import numpy as np
import pandas as pd
import csv
import scipy.stats as st

In [374]:


# Specify the path to your CSV file
csv_file_path = 'D:\Artificial intelligence\IA 2023 -\Classification\iris\iris.data'

# Initialize an empty array to store the data
data_array = []

# Open the CSV file
with open(csv_file_path, 'r') as csv_file:
    # Create a CSV reader
    csv_reader = csv.reader(csv_file)
    
    # Iterate through each row in the CSV file
    for row in csv_reader:
        converted_row = [float(value) if value.replace('.', '', 1).isdigit() else value for value in row]#loop through each element of the row
        # Append each row to the data array
        data_array.append(converted_row)

# Now, data_array contains the data from the CSV file

df= pd.DataFrame(data_array)
df.head(len(df))



df.rename(columns={0:'sepal length',1:'sepal width',2:'petal length',3:'petal width ',4:'class'})





,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


# Coding target

In [375]:
new_array=[]
values_to_keep =['Iris-setosa','Iris-virginica']
for row in data_array:
   if row[4] in values_to_keep:
      new_array.append(row)
for i in range (len(new_array)):
   if new_array[i][4] == 'Iris-setosa':
      new_array[i][4] = 1
   else :
      new_array[i][4]=0
new_array=np.array(new_array)
target= np.array(new_array[:, -1:])




In [376]:
new_df=pd.DataFrame(new_array)
corr =  new_df.corr()
new_array = np.delete(new_array, -1, axis=1)
print(corr)
new_df.head()

          0         1         2         3         4
0  1.000000 -0.206766  0.904796  0.853251 -0.840970
1 -0.206766  1.000000 -0.478845 -0.445980  0.536251
2  0.904796 -0.478845  1.000000  0.969650 -0.980933
3  0.853251 -0.445980  0.969650  1.000000 -0.974207
4 -0.840970  0.536251 -0.980933 -0.974207  1.000000


,0,1,2,3,4
0,5.1,3.5,1.4,0.2,1.0
1,4.9,3.0,1.4,0.2,1.0
2,4.7,3.2,1.3,0.2,1.0
3,4.6,3.1,1.5,0.2,1.0
4,5.0,3.6,1.4,0.2,1.0


#### Correlation between features and output are greater than |0.3|, so there is no reason to delete
#### Some of features are highly correlated with others, in most of the cases we would've deleted one of them but we don't have much data neither features so the processing will be quick either way

In [377]:
new_df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

setosa_std=round(new_df[:50].std(axis="rows"),2)
setosa_mean=round(new_df[:50].mean(axis="rows"),2)

versicolour_std=round(new_df[50:100].std(axis="rows"),2)
versicolour_mean=round(new_df[50:100].mean(axis="rows"),2)




testing_data1 = np.column_stack([
    np.round(st.norm(loc=mean, scale=std).rvs(size=50),2) for mean, std in zip(setosa_mean, setosa_std)
])
testing_data2 = np.column_stack([
    np.round(st.norm(loc=mean, scale=std).rvs(size=50),2) for mean, std in zip(versicolour_mean, versicolour_std)
])


testing_data=np.concatenate((testing_data1,testing_data2))


## Classification class with gradient descent

In [378]:
class Classification ():
    def __init__(self,features,output) -> None:
        self.n=np.shape(features)[1]
        self.m=np.shape(features)[0]
        self.weights = np.zeros((self.n))
        self.bias=0
        self.X = features
        self.outputs = output
        self.alpha = 0.1

    def gradient_descent(self,epochs):
        W=self.weights
        b=self.bias
        alpha = self.alpha
 
        for i in range (epochs): 

            z=np.dot(self.X,(W))+b
            y_hat=self.sigmoid(z)
            

            dc_dwj=[]
            for j in range (self.n):
                sum=0
                for i in range(self.m):
                    sum+= self.X[i,j]*(y_hat[i]-self.outputs[i])
                sum/=self.m
             
                dc_dwj.append(sum)

            sum_bias = 0
            for i in range(self.m):
                    sum_bias+= (y_hat[i]-self.outputs[i])
            sum_bias/=self.m
            b = b - alpha* sum_bias

            for i in range (self.n):
                W[i]=W[i]- alpha*dc_dwj[i]



        return W,b

    def sigmoid(self,z):

        
        return 1/(1+np.exp(-z))

    



## Training

In [379]:
features = new_array
array_output = target


cf = Classification(features,array_output)

W,b=cf.gradient_descent(50)

## Testing

In [380]:
res = np.dot(testing_data,(W))+b

res = cf.sigmoid(res)

for i in range(len(res)):
    if res[i]>0.5:
        res[i]=1
    else :
        res[i]=0
df_res=pd.DataFrame(res)
fails=0
for idx,value in enumerate(res):
    if value!= target[idx]:
        fails+=1

print(f'Accuracy : {100-fails*100/100}% ')
df_res.head(len(res))


Accuracy : 100.0% 


,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
95,0.0
96,0.0
97,0.0
98,0.0
